Autenticação e Inicialização da biblioteca do Earth Engine.

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

Carregando uma coleção de imagens.

In [ ]:
imagem = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
  .filterDate('2019-01-01','2020-01-01')\
  .filterBounds(ee.Geometry.Point(-49.459937, -28.5919285))

print("Quantidade de Imagens: ", imagem.size().getInfo())

Quantidade de Imagens:  21


Função para visualizar mapas interativos no Google Colab (Fonte: [Cesar Aybar](https://github.com/csaybar/EEwPython)).

In [ ]:
import folium

def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

Visualizando nosso mapa.

In [ ]:
mediana = imagem.median()

dicionario = {
    'Mediana' : mediana.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000})
}

centro = [-28.5919285, -49.459937]
Mapdisplay(centro, dicionario, zoom_start= 12)

Função para obter lista de datas disponíveis em uma Image Collection (Fonte: [Vincent](https://gis.stackexchange.com/questions/284826/unique-list-of-image-dates-from-image-collection-in-google-earth-engine)).

In [ ]:
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())

Visualizando datas disponíveis.

In [ ]:
print('Datas Disponíveis: ', ymdList(imagem))

Datas Disponíveis:  ['20190128', '20190213', '20190301', '20190402', '20190418', '20190504', '20190520', '20190605', '20190621', '20190707', '20190723', '20190808', '20190824', '20190909', '20190925', '20191011', '20191027', '20191112', '20191128', '20191214', '20191230']


Carregando uma imagem LANDSAT.

In [ ]:
imagem_unica = ee.Image('LANDSAT/LC08/C01/T1/LC08_220080_20191230')

print('Bandas: ', imagem_unica.bandNames().getInfo())
print('Cobertura de Nuvens (%): ', imagem_unica.get('CLOUD_COVER').getInfo())

dicio2 = imagem_unica.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 30000})

Mapdisplay(centro, {'LANDSAT': dicio2})

Bandas:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']
Cobertura de Nuvens (%):  3.9800000190734863


Feições.

In [ ]:
ponto = ee.Geometry.Point([-49.4, -28.6])
linha = ee.Geometry.LineString([[-49.4, -28.6], [-49.0, -28.6], [-49.0, -28.2]])
poligono = ee.Geometry.Polygon([[-49.4, -28.6], [-49.0, -28.6], [-49.0, -28.2], [-49.4, -28.6]]);

ponto_info = ponto.getInfo()
linha_info = linha.getInfo()
poligono_info = poligono.getInfo()

Mapdisplay(centro, {'Ponto': ponto_info, 'Linha': linha_info, 'Polígono': poligono_info})

NDVI

In [ ]:
NDVI = imagem_unica.expression(
      '(NIR - RED) / (NIR + RED)', {
          'RED': imagem_unica.select('B4'), 
          'NIR': imagem_unica.select('B5')})

NDVI_mapa = {'min': 0, 'max': 1, 'palette': ['FF0000', '00FF00']}
visualizacao = {"NDVI": NDVI.getMapId(NDVI_mapa)}

Mapdisplay(centro, visualizacao)